<a href="https://colab.research.google.com/github/abdallaho98/AutoSpeech/blob/master/NB_and_Logistical_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install ktrain

     |████████████████████████████████| 25.2MB 1.7MB/s 
     |████████████████████████████████| 983kB 44.9MB/s 
     |████████████████████████████████| 245kB 45.4MB/s 
     |████████████████████████████████| 552kB 48.4MB/s 
     |████████████████████████████████| 3.7MB 30.5MB/s 
     |████████████████████████████████| 870kB 43.7MB/s 
     |████████████████████████████████| 1.0MB 39.6MB/s 
  Created wheel for ktrain: filename=ktrain-0.12.3-cp36-none-any.whl size=25228500 sha256=3a0a59e13da1c7e01236d9d9aabaf54182806028d4b2149a1d0e4b0e732ca579
  Stored in directory: /root/.cache/pip/wheels/93/8f/77/a036a53942ede8e46e333d39b6abd9aaf3ab91aaf96d83e70f
  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=a6f30fc85f817a57009e76e34f1a50eaf1b3df7c5c6186a5d0513d58e5fc8c12
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl siz

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [0]:
import ktrain
from ktrain import text

In [9]:
import pandas as pd
filepath = 'legalts.txt'
df = pd.DataFrame(columns=('rating', 'review'))
with open(filepath) as fp:
   line = fp.readline()
   cnt = 0
   while line:
       x = line.strip().split("  ")
       df.loc[cnt] = [int(x[1]) , x[0]]
       line = fp.readline()
       cnt += 1
df['rating'] = df['rating'].apply(lambda x: 0.0 if x < 3 else 1.0)
df.head()

,rating,review
0,1.0,يمكن أن ينصب الفحص الطبي على السوابق الوراثية ...
1,1.0,يمكن للمكتب، بقطع النظر عن أحكام المادة 4 أعلا...
2,1.0,(يمكن اكتساب الجنسية الجزائرية بالزواج من جزائ...
3,1.0,يمكن إنشاء فروع في دائرة اختصاص المحاكم على مس...
4,1.0,يمكن أحد الطرفين أن يفسخ العقد في أي وقت شريطة...


In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df['review'].values, 
                 df['rating'].values,        
                 test_size=0.2)

In [0]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [0]:
vect = CountVectorizer(tokenizer=tokenize)
tf_train = vect.fit_transform(X_train)
tf_test = vect.transform(X_test)

In [32]:
model = LogisticRegression(C=0.2, dual=False)
model.fit(tf_train, y_train)
preds = model.predict(tf_test)
accuracy = (preds == y_test).mean()
print(accuracy)

0.7272727272727273


In [33]:
p = tf_train[y_train==1].sum(0) + 1
q = tf_train[y_train==0].sum(0) + 1
r = np.log((p/p.sum()) / (q/q.sum()))
b = np.log(len(p) / len(q))
pre_preds = tf_test @ r.T + b
preds = pre_preds.T > 0
accuracy = (preds == y_test).mean()
print(accuracy)

0.8181818181818182


In [39]:
vect = TfidfVectorizer(strip_accents='unicode', tokenizer=tokenize, ngram_range=(1, 2), max_df=0.9, min_df=3, sublinear_tf=True)
tfidf_train = vect.fit_transform(X_train)
tfidf_test = vect.transform(X_test)
model = LogisticRegression(C=30, dual=False)
model.fit(tfidf_train, y_train)
preds = model.predict(tfidf_test)
accuracy = (preds==y_test).mean()
print(accuracy)

0.8181818181818182
